# Дашборд Кейса №2

## Импорты библиотек

In [1]:
from typing import Tuple, Optional, List, Dict
import pandas as pd
import gradio as gr
import os
import re
import plotly.express as px

gr.close_all()


/Users/m/edu/mipt/practicum/team-8-case-2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Вспомогательные функции

Подгрузка csv файла

In [2]:
def load_data(file):
    """
    Загружает CSV-файл в DataFrame.
    
    Параметры
    ---------
    file : UploadedFile или None
        Объект загруженного файла с атрибутом `name`.
    
    Возвращает
    ----------
    tuple
        (DataFrame, preview, status_message, description)
    """
    if file is None:
        return None, pd.DataFrame(), "Ошибка. Загрузите файл в формате CSV", None
    
    try:
        df = pd.read_csv(
            file.name, 
            parse_dates=["survey_creation_dt", "survey_response_dt"]
        )
    except Exception as e:
        return None, pd.DataFrame(), f"Ошибка чтения файла: {e}", None
    
    filename = os.path.basename(file.name)
    status = f"Файл {filename} успешно загружен"
    description = f"Файл содержит {len(df):,} строк и {len(df.columns)} столбцов."
    
    return df, df.head(10), status, description


Очистка данных

In [3]:
def clear_data(df):
    """
    Выполняет предобработку и очистку данных анкетного опроса.
    
    Этапы:
    1. Вычисление времени ответа в днях
    2. Исправление перепутанных дат (ответ раньше создания)
    3. Заполнение пропусков в language значением 'RU'
    
    Параметры
    ---------
    df : pd.DataFrame или None
        Исходный DataFrame с колонками survey_creation_dt, survey_response_dt, language.
    
    Возвращает
    ----------
    tuple
        (cleaned_df, preview, status_message, description)
    """
    if df is None:
        return None, pd.DataFrame(), "Ошибка. Сначала загрузите файл CSV.", None

    # 1. Вычисление времени ответа
    df["response_time_days"] = (
        df["survey_response_dt"] - df["survey_creation_dt"]
    ).dt.total_seconds() / (24 * 3600)

    # 2. Исправление перепутанных дат
    negative_before = (df["response_time_days"] < 0).sum()
    
    swap_mask = df["response_time_days"] < 0
    df["dates_swapped"] = swap_mask
    df.loc[swap_mask, ["survey_creation_dt", "survey_response_dt"]] = df.loc[
        swap_mask, ["survey_response_dt", "survey_creation_dt"]
    ].values

    # Пересчёт времени ответа
    df["response_time_days"] = (
        df["survey_response_dt"] - df["survey_creation_dt"]
    ).dt.total_seconds() / (24 * 3600)

    negative_after = (df["response_time_days"] < 0).sum()

    # 3. Заполнение пропусков в language
    language_missing = df["language"].isnull().sum()
    df["language"] = df["language"].fillna("RU")

    # Формирование отчёта
    description = (
        f"Записей с отрицательным временем ответа: {negative_before}.\n"
        f"После исправления: {negative_after}.\n"
        f"Заполнено пропусков в language: {language_missing}.\n"
        f"Итого: {len(df):,} строк и {len(df.columns)} столбцов."
    )

    return df, df.head(10), "Данные успешно обработаны", description


Функция создания

In [4]:
# Разрешённые колонки для анализа
ALLOWED_COLS = [
    "csat_level",
    "age",
    "gender",
    "user_income",
    "response_time_days",
]

ALLOWED_COLS_CSAT = [
    "age",
    "gender",
    "user_income",
    "response_time_days",
]


In [5]:
def update_col_dropdown_with_stats(df, all_cols: List[str]) -> gr.Dropdown:
    """
    Создаёт выпадающий список с названиями колонок и количеством уникальных значений.
    
    Формат: "col_name (n_unique)"
    """
    if df is None:
        return gr.Dropdown(choices=[], value=None)

    cols = [c for c in all_cols if c in df.columns]
    unique_counts = df[cols].nunique(dropna=False)
    
    choices = [f"{col} ({int(unique_counts[col])})" for col in cols]
    value = choices[0] if choices else None
    return gr.Dropdown(choices=choices, value=value)


def _parse_choice(choice_text: str) -> Tuple[Optional[str], Optional[int]]:
    """
    Извлекает имя колонки и число из строки формата "col_name (123)".
    
    Возвращает
    ----------
    tuple
        (column_name, count) или (None, None) если парсинг не удался
    """
    if not choice_text:
        return None, None
    
    match = re.match(r"^(.*)\s\((\d+)\)$", choice_text)
    if match:
        return match.group(1), int(match.group(2))
    return choice_text, None


def cat_counts_for_choice(df, choice_text: str, use_share: bool) -> pd.DataFrame:
    """
    Возвращает частоты или доли значений для указанного столбца.
    
    Параметры
    ---------
    df : pd.DataFrame
        Исходные данные
    choice_text : str
        Текст выбора формата "col_name (n)"
    use_share : bool
        True — вернуть доли, False — абсолютные частоты
    
    Возвращает
    ----------
    pd.DataFrame
        С колонками 'category' и 'value'
    """
    col, top_k = _parse_choice(choice_text)
    
    if df is None or col is None or col not in df.columns:
        return pd.DataFrame({"category": [], "value": []})

    series = df[col].astype(str).fillna("NaN")
    value_counts = series.value_counts(normalize=use_share, dropna=False)

    if top_k is None:
        top_k = int(series.nunique(dropna=False))

    result = value_counts.head(top_k).rename_axis("category").reset_index(name="value")
    result["value"] = pd.to_numeric(result["value"], errors="coerce")
    return result


def update_plot_count_share(df, choice_text: str, mode: str):
    """
    Обновляет данные графика в зависимости от режима (доли/абсолютные значения).
    """
    if mode is None:
        mode = "count"

    use_share = (mode == "share")
    y_title = "Доля наблюдений" if use_share else "Количество наблюдений"
    
    df_out = cat_counts_for_choice(df, choice_text, use_share=use_share)
    return gr.update(value=df_out, y_title=y_title)


In [6]:
def build_interval_rank(values) -> Tuple[Dict[str, int], List[str]]:
    """
    Строит порядок для категорий-интервалов вида "<5", "10-20", ">100".
    
    Возвращает
    ----------
    tuple
        (rank_dict, ordered_list)
    """
    unique_vals = [str(v).strip() for v in set(values) if v is not None]

    lt_vals = sorted([v for v in unique_vals if "<" in v])
    gt_vals = sorted([v for v in unique_vals if ">" in v])
    mid_vals = [v for v in unique_vals if "<" not in v and ">" not in v]

    def parse_range(s: str) -> Tuple[int, int]:
        s_clean = s.replace(" ", "")
        match = re.match(r"^(\d+)-(\d+)$", s_clean)
        if not match:
            return (10**18, 10**18)
        return (int(match.group(1)), int(match.group(2)))

    mid_sorted = sorted(mid_vals, key=parse_range)

    # Построение рангов
    rank = {}
    for v in lt_vals:
        rank[v] = 0
    for i, v in enumerate(mid_sorted, start=1):
        rank[v] = i
    max_rank = len(mid_sorted)
    for v in gt_vals:
        rank[v] = max_rank + 1

    order = lt_vals + mid_sorted + gt_vals
    return rank, order


In [7]:
def mean_csat_by_param(df, param_col: str) -> Tuple[pd.DataFrame, List[str]]:
    """
    Вычисляет среднее CSAT по группам из указанного столбца.
    
    Возвращает
    ----------
    tuple
        (DataFrame с group и mean_csat, список порядка групп)
    """
    col, _ = _parse_choice(param_col)

    if df is None or "csat_level" not in df.columns or col not in df.columns:
        return pd.DataFrame({"group": [], "mean_csat": []}), []

    tmp = df[[col, "csat_level"]].dropna().copy()
    tmp[col] = tmp[col].astype(str)

    grouped = tmp.groupby(col)["csat_level"].mean()
    
    _, order = build_interval_rank(tmp[col].unique())
    order = [str(x) for x in order]

    grouped = grouped.reindex(order).dropna()

    result = grouped.rename("mean_csat").reset_index().rename(columns={col: "group"})
    result["group"] = result["group"].astype(str)
    return result, order


def update_mean_plot(df, param_col: str):
    """
    Создаёт столбчатый график среднего CSAT по группам.
    """
    data, order = mean_csat_by_param(df, param_col)

    if data is None or data.empty:
        fig = px.bar(
            pd.DataFrame({"group": [], "mean_csat": []}),
            x="group", y="mean_csat"
        )
        return fig

    fig = px.bar(
        data,
        x="group",
        y="mean_csat",
        text_auto=".2f",
        labels={"group": "Группа", "mean_csat": "Средняя оценка"},
    )

    fig.update_xaxes(categoryorder="array", categoryarray=order)
    fig.update_yaxes(range=[1, 5.1])
    fig.update_traces(marker_color="orange", textposition="outside")
    fig.update_layout(
        plot_bgcolor="black",
        paper_bgcolor="black",
        font_color="white",
        title_text="Зависимость оценки от параметра",
        title_x=0.5,
    )
    return fig


## Структура дашборда

In [ ]:
# Структура дашборда
with gr.Blocks(title="Кейс 2 - Команда 8", theme=gr.themes.Soft()) as case2:
    gr.Markdown("""
    <div style="text-align:center">
    
    # Дашборд команды №8  
    Кейс №2. Пенсионный фонд — анализ удовлетворённости пользователей.
    
    </div>
    """)
    
    # State для хранения данных между вкладками
    df_state = gr.State(None)
    cols_state = gr.State(ALLOWED_COLS)
    cols_state_csat = gr.State(ALLOWED_COLS_CSAT)

    with gr.Tabs():
        # === Вкладка загрузки данных ===
        with gr.Tab("Загрузка данных"):
            with gr.Row():
                result_box = gr.Textbox(label="Статус", lines=2, scale=1)
                description_box = gr.Textbox(label="Описание", lines=2, scale=3)
            
            file_input = gr.File(
                label="Выберите CSV-файл", 
                file_types=[".csv"], 
                height=100
            )
            
            with gr.Row():
                load_button = gr.Button("Загрузить файл", variant="primary")
                clear_button = gr.Button("Очистить данные", variant="secondary")
            
            head_df = gr.DataFrame(
                label="Предпросмотр данных",
                interactive=False,
                wrap=True,
            )

        # === Вкладка базовых гистограмм ===
        with gr.Tab("Гистограммы"):
            with gr.Row():
                col_base = gr.Dropdown(
                    label="Столбец", 
                    choices=[], 
                    value=None, 
                    interactive=True
                )
                mode_base = gr.Radio(
                    choices=[
                        ("Доли", "share"),
                        ("Количество", "count"),
                    ],
                    value="count",
                    label="Режим",
                )

            plot_base = gr.BarPlot(
                value=pd.DataFrame({"category": [], "value": []}),
                x="category",
                y="value",
                y_title="Количество",
                x_title="Категория",
                title="Распределение по столбцу",
            )

        # === Вкладка CSAT по параметрам ===
        with gr.Tab("CSAT ~ параметр"):
            param_col_csat = gr.Dropdown(
                choices=["age", "user_income", "gender"],
                value="age",
                label="Параметр для анализа",
            )
            mean_plot_csat = gr.Plot(label="Зависимость оценки от параметра")

        # === Вкладка авторов ===
        with gr.Tab("Авторы"):
            with gr.Row():
                gr.Image(value="team/team_01.png", height=250, width=50, show_label=False)
                gr.Image(value="team/team_02.png", height=250, width=50, show_label=False)
                gr.Image(value="team/team_03.png", height=250, width=50, show_label=False)
            with gr.Row():
                gr.Markdown("""
                    **Ольга Ожерельева**  
                """)
                gr.Markdown("""
                    **Кирилл Никулин**  
                """)
                gr.Markdown("""
                    **Максим Смирнов**  
                """)

    # === Обработчики событий ===
    
    # Загрузка данных
    load_evt = load_button.click(
        load_data,
        inputs=[file_input],
        outputs=[df_state, head_df, result_box, description_box],
    )
    load_evt.then(
        update_col_dropdown_with_stats,
        inputs=[df_state, cols_state],
        outputs=[col_base],
    )
    load_evt.then(
        update_plot_count_share,
        inputs=[df_state, col_base, mode_base],
        outputs=plot_base,
    )
    load_evt.then(
        update_col_dropdown_with_stats,
        inputs=[df_state, cols_state_csat],
        outputs=[param_col_csat],
    )
    load_evt.then(
        update_mean_plot,
        inputs=[df_state, param_col_csat],
        outputs=mean_plot_csat,
    )

    # Очистка данных
    clear_evt = clear_button.click(
        clear_data,
        inputs=[df_state],
        outputs=[df_state, head_df, result_box, description_box],
    )
    clear_evt.then(
        update_col_dropdown_with_stats,
        inputs=[df_state, cols_state],
        outputs=[col_base],
    )
    clear_evt.then(
        update_plot_count_share,
        inputs=[df_state, col_base, mode_base],
        outputs=plot_base,
    )
    clear_evt.then(
        update_col_dropdown_with_stats,
        inputs=[df_state, cols_state_csat],
        outputs=[param_col_csat],
    )
    clear_evt.then(
        update_mean_plot,
        inputs=[df_state, param_col_csat],
        outputs=mean_plot_csat,
    )

    # Обновление графиков при изменении параметров
    col_base.change(
        update_plot_count_share,
        inputs=[df_state, col_base, mode_base],
        outputs=plot_base,
    )
    mode_base.change(
        update_plot_count_share,
        inputs=[df_state, col_base, mode_base],
        outputs=plot_base,
    )
    param_col_csat.change(
        update_mean_plot,
        inputs=[df_state, param_col_csat],
        outputs=mean_plot_csat,
    )


/var/folders/c0/h9v3_pyd6nz79fvw55tfy6980000gn/T/ipykernel_11782/2474732286.py:2: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(title="Кейс 2 - Команда 8", theme=gr.themes.Soft()) as case2:


In [9]:
case2.launch(
    prevent_thread_lock=True, 
    inbrowser=True, 
    share=False, 
    show_error=True,
)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
